In [3]:
%%capture
# Capture because braindecode module prints a lot :S
from pyFiles.training import *
from pyFiles.models import *
from pyFiles.dataprep_down import *
from pyFiles.dataprep2_down import *
#
import torch
from torch import nn
from torch.nn import functional as F
#
from pyFiles.losses import InfoNceLoss, VICReg, BarlowTwins

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2" 

In [ ]:
#PREP DENUEVO
prep("dataverse_files", mode = "per_win", save = True, save_dir = "data_down")

In [ ]:
spliter = DFSpliter(save = True, mode= "down_win")
spliter("down_prep_windows.csv",".")

In [ ]:
transforms = Augmentations(5, False, AUGMENTATIONS)

In [ ]:
train_dataset = CustomEEGDataset("down_win_train_feats.csv", root_dir = "", multi = False, transform = transforms)
val_dataset = CustomEEGDataset("down_win_val_feats.csv", root_dir = "", multi = False)

In [ ]:
trainloader = DataLoader(train_dataset, batch_size = 1000, shuffle = True, num_workers = 2)
valloader = DataLoader(val_dataset, batch_size = 1000, shuffle = False, num_workers = 2)

In [ ]:
model = SleepStager(n_chans = 19, pretext = False)
enc, fc = torch.load("multis/infonce/bestEncoderParamsmulti_inf.pt")
model.encoder.load_state_dict(enc)

criterion = nn.CrossEntropyLoss()

#FREEZE
for param in model.encoder.parameters():
  param.requires_grad = False

optimizer = torch.optim.Adam(model.final.parameters(), lr = 0.001)
downtrain(model, 50, trainloader, valloader, criterion, optimizer, state = None, name = "multi_inf")